In [5]:
import keras
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,precision_score, precision_recall_curve, recall_score, roc_curve, auc, confusion_matrix,classification_report
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
!ls -al data/

totale 362896
drwxrwxr-x 2 ec2-user ec2-user     4096 16 gen 22.37 .
drwxrwxr-x 4 ec2-user ec2-user     4096 16 gen 22.37 ..
-rw-rw-r-- 1 ec2-user ec2-user 93295897 16 gen 22.37 demo-g1.txt
-rw-rw-r-- 1 ec2-user ec2-user 14890433 16 gen 22.37 demo-g1-v2.txt
-rw-rw-r-- 1 ec2-user ec2-user 86071971 16 gen 22.37 demo-g3.txt
-rw-rw-r-- 1 ec2-user ec2-user  5459634 16 gen 22.37 demo-g3-v2-test.txt
-rw-rw-r-- 1 ec2-user ec2-user 13160539 16 gen 22.37 demo-g3-v2.txt
-rw-rw-r-- 1 ec2-user ec2-user    50590 16 gen 22.37 em_survey_choice_ids.json
-rw-rw-r-- 1 ec2-user ec2-user 69299446 16 gen 22.37 spark-demo-g1.json
-rw-rw-r-- 1 ec2-user ec2-user 44667946 16 gen 22.37 spark-demo-g3.json
-rw-rw-r-- 1 ec2-user ec2-user 44353740 16 gen 22.37 spark-demo-g3-v2.json
-rw-rw-r-- 1 ec2-user ec2-user   324420 16 gen 22.37 url_entities.json


In [6]:
data = load_svmlight_file('data/demo-g3-v2.txt')
X,y=data[0],data[1]
X_s,y_s=resample(X,y,n_samples=100000,replace=False)


In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_e=le.fit_transform(y_s)

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X_s,y_e,test_size=.3,random_state=42)

In [9]:
model = keras.Sequential()

In [11]:
from keras.layers import Dense

In [12]:
np.unique(y_e)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [54]:
model.add(Dense(128,activation='relu',input_shape=(X_train.shape[1],)))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(11,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [55]:
model.fit(X_train,y_train,epochs=10,batch_size=64,validation_data=(X_test,y_test))

Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 21s 297us/step - loss: 1.9650 - acc: 0.2917 - val_loss: 2.3585 - val_acc: 0.2063
Epoch 2/10
70000/70000 [==============================] - 19s 271us/step - loss: 1.9354 - acc: 0.3062 - val_loss: 2.4163 - val_acc: 0.2032
Epoch 3/10
70000/70000 [==============================] - 19s 275us/step - loss: 1.9362 - acc: 0.3046 - val_loss: 2.3355 - val_acc: 0.2030
Epoch 4/10
70000/70000 [==============================] - 19s 275us/step - loss: 1.9305 - acc: 0.3066 - val_loss: 2.3659 - val_acc: 0.2061
Epoch 5/10
70000/70000 [==============================] - 19s 274us/step - loss: 1.9232 - acc: 0.3094 - val_loss: 2.3565 - val_acc: 0.1979
Epoch 6/10
70000/70000 [==============================] - 19s 274us/step - loss: 1.9292 - acc: 0.3055 - val_loss: 2.3244 - val_acc: 0.2022
Epoch 7/10
70000/70000 [==============================] - 19s 274us/step - loss: 1.9252 - acc: 0.3088 - val_loss: 2.

In [43]:
model.evaluate(X_test,y_test)

30000/30000 [==============================] - 3s 93us/step


[2.3162487575531006, 0.2047]

In [44]:
y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred,axis=1)

In [45]:
def model_eva(model,X_test,y_test):
    y_test_pred=model.predict(X_test)
    y_test_pred=np.argmax(y_test_pred,axis=1)
    print('precision_score:',precision_score(y_test,y_test_pred,average='macro'))
    print(confusion_matrix(y_test,y_test_pred).T)
    print(np.unique(y_test,return_counts=True))
    print(np.unique(y_test_pred,return_counts=True))
    print(classification_report(y_test,y_test_pred))
    print("******************************************************************")

In [56]:
model_eva(model,X_test,y_test)

precision_score: 0.141861912475433
[[ 605  388  439  354  333  263  190  148  112   77  163]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [ 209  162  255  271  262  214  168  135   84   68   99]
 [ 101  116  328  368  346  245  153  123   83   64   80]
 [ 169  144  419  582  565  587  446  291  191  162  181]
 [ 218  174  556  932 1168 1234  970  781  571  433  643]
 [  25   19   71  136  200  243  195  165  128  119  175]
 [  12    8   27   26   59   60   58   70   49   41   63]
 [   4    0   11   21   14   21   21   20   13   18   26]
 [  14    6   20   33   43   50   44   51   60   32   94]
 [ 148  137  338  464  708  844  803  937  936  927 2770]]
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([1505, 1154, 2464, 3187, 3698, 3761, 3048, 2721, 2227, 1941, 4294]))
(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([3072, 1927, 2007, 3737, 7680, 1476,  473,  169,  447, 9012]))
             precision    recall  f1-score   support

          0       0.20  

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
np.unique(y_pred,return_counts=True)

(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([3551, 2386, 1161, 3414, 7035, 1960, 1189,  327,  153, 8824]))

In [48]:
np.unique(y_test,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([1505, 1154, 2464, 3187, 3698, 3761, 3048, 2721, 2227, 1941, 4294]))

In [53]:
model_eva(model,X_train,y_train)

precision_score: 0.31771046023522365
[[2372 1620 1424  841  777  602  424  292  182  143  270]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [ 261  215 1220  966  496  331  193  126   96   67  115]
 [  33   86  731 1713 1164  406  182  100   74   26   21]
 [ 396  456 1365 1901 3173 2611 1610 1116  745  536  652]
 [  69   59  440  920 1491 2569 1337  724  530  349  332]
 [  39   44  139  308  501  622 1525  997  460  297  291]
 [  11   12   16   41   54   91  314  841  447  213  128]
 [   2    1    0    2    2    7   15   56  284  212   49]
 [   0    0    0    3    2    6    9   21  108  262   93]
 [ 227  182  537  734 1068 1357 1562 1895 2227 2654 8082]]
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 3410,  2675,  5872,  7429,  8728,  8602,  7171,  6168,  5153,
        4759, 10033]))
(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 8947,  4086,  4536, 14561,  8820,  5223,  2168,   630,   504,
       20525]))
             precision    recall  f1-scor

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
